In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import json

pd.set_option('display.max_rows', None)

In [2]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('client-secret.json', scope)
client = gspread.authorize(creds)
sh = client.open('Fantasy Cricket')

In [3]:
database_sheet = sh.worksheet("IPL Player Database")

In [4]:
URL = "https://www.espncricinfo.com/ci/content/squad/index.html?object=1210595"
page = requests.get(URL)
bs = BeautifulSoup(page.content, 'lxml')

In [5]:
players = {}
columns = []
ultag = bs.find('ul', {'class': 'squads_list'})
for litag in ultag.find_all('li'):
    a = litag.find('a', href=True)
    team = " ".join(a.text.split()[:-1])
    print(team)
    href = a['href']
    squad_URL = "https://www.espncricinfo.com/" + href
    squad_page = requests.get(squad_URL)
    squad_bs = BeautifulSoup(squad_page.content,'lxml')
    players_div  = squad_bs.find('div', {'role': 'main'})
    player_list = players_div.find_all("ul")[0]
    for player in player_list.find_all("li"):
        for atag in player.find_all('a', href=True):
            name = atag.text.strip()
            href = atag['href']
        print(name)
        players[name] = []
        player_attributes = player.find_all('span')
        
        # Overseas Player
        if len(player_attributes) == 5:
            players[name].append("Overseas")
        else:
            players[name].append("Domestic")

        # Role
        role_list = [span for span in player_attributes if span.find('b') and "Playing role:" in span.text]
        if (len(role_list) == 0):
            players[name].append("Unknown")
        else:
            role = role_list[0].text.split(':')[-1]
            if "allrounder" in role or "Allrounder" in role:
                players[name].append("All-Rounder")
            elif 'batsman' in role or 'Batsman' in role:
                players[name].append("Batsman")
            elif 'Bowler' in role:
                players[name].append("Bowler")

        # Commentary Name
        
        player_URL = "https://www.espncricinfo.com/" + href

        player_page = requests.get(player_URL)
        player_bs = BeautifulSoup(player_page.content,'lxml')
        
        matches_table = player_bs.find("div", {"class": "ciPlayertextbottomborder"}).find_next_sibling('table')
        match_href = matches_table.find("a", href=True)['href']
        
        
        match_URL = "https://www.espncricinfo.com/" + match_href
        
        match_page = requests.get(match_URL)
        match_bs = BeautifulSoup(match_page.content,'lxml')
        
        table_body=match_bs.find_all('tbody')
        xis = []
        table_foot =match_bs.find_all('tfoot')

        for i, table in enumerate(table_foot):
            rows = table.find_all('tr')
            if len(rows) == 3:
                dnb_names = rows[1].find_all('span')
                dnb_names = [x.text.replace(u'\xa0', '').strip(' ,') for x in dnb_names]
                dnb_names = [re.sub(r"\W+", ' ', i.split("(c)")[0]).strip() for i in dnb_names]
                dnb_names = [i for i in dnb_names if i]
                xis.extend(dnb_names)

        for i, table in enumerate(table_body):
            rows = table.find_all('tr')
            for row in rows:
                cols=row.find_all('td')
                cols=[x.text.strip() for x in cols]
                if len(cols) >= 8:
                    player_entry = re.sub(r"\W+", ' ', cols[0].split("(c)")[0]).strip()
                    if name.split()[-1] in player_entry and len(players[name]) != 3:
                        players[name].append(player_entry)
                if len(cols) == 2:
                    if cols[1] == 'Batsman' or cols[1] == 'Allrounder' or cols[1] == 'Bowler' or cols[1] == 'Wicketkeeper':
                        player_entry = re.sub(r"\W+", ' ', cols[0].split("(c)")[0]).strip()
                        if name.split()[-1] in player_entry and len(players[name]) != 3:
                            players[name].append(player_entry)
                if (len(players[name]) == 3):
                    break
            if (len(players[name]) == 3):
                break

                            
        break
    break
columns.append("Overseas/Domestic")
columns.append("Role")
columns.append("Commentary Name")


# display(pd.DataFrame.from_dict(players, orient='index', columns=columns))

Chennai Super Kings
MS Dhoni


In [6]:
display(pd.DataFrame.from_dict(players, orient='index', columns=columns))

,Overseas/Domestic,Role,Commentary Name
MS Dhoni,Overseas,Batsman,MS Dhoni


,Overseas/Domestic,Role,Commentary Name
KM Asif,Domestic,Bowler,KM Asif


In [22]:
players_li = [[name] + players[name] for name in players.keys()]
database_sheet.update([["Name", "Overseas/Domestic", "Role" "Commentary Name"]] + players_li)

{'spreadsheetId': '1vCCQ418PRSuohwiLEs8ewKIrhhrIXoTpG3LiKv8kYeo',
 'updatedRange': "'IPL Player Database'!A1:D2",
 'updatedRows': 2,
 'updatedColumns': 4,
 'updatedCells': 7}

In [16]:
players

{'KM Asif': ['Domestic', 'Bowler', 'KM Asif']}